In [1]:
#Let's try to make these calls faster
import tekore as tk
import asyncio
import yaml
import requests


In [2]:
credentials_path = "credentials.yaml"
with open(credentials_path) as file:
    cred = yaml.load(file, Loader=yaml.FullLoader)

In [3]:
conf = cred['client_id'], cred['client_secret']

app_token = tk.request_client_token(*conf)

#Let's try to connect these words
spotify = tk.Spotify(app_token)


In [4]:
app_token.access_token

'BQChUZzYF-CQb4_t5XJ4wVQSeFA0xGNfGV-I4zqTyEcpiYi7yGa7egeYf1u1FxRUeV5oOOjEy6LlXkx3LM0'

## Playlist search

In [8]:
obj = spotify.search("Top 50 USA", types=("playlist",), limit=1)

In [9]:
ids = []
for track in spotify.playlist_items(obj[0].items[0].id, as_tracks=True)['items']:
    ids.append(track["track"]["id"])


In [10]:
features = spotify.tracks_audio_features(ids)

In [11]:
type(features)

tekore.model.ModelList

## Track search

In [9]:
obj = spotify.search("Show must go on", types=("track",), limit=1)

In [10]:
obj[0].items[0].id

'3m86qx7aIeqFcYcQ1UXUvP'

In [11]:
features = spotify.track_audio_features(obj[0].items[0].id)

In [107]:
features

AudioFeatures with fields:
  acousticness = 0.526
  analysis_url = 'https://api.spotify.com/v1/audio-analysis/3m86qx7aIeq...'
  danceability = 0.421
  duration_ms = 272107
  energy = 0.517
  id = '3m86qx7aIeqFcYcQ1UXUvP'
  instrumentalness = 1.82e-06
  key = 11
  liveness = 0.136
  loudness = -11.49
  mode = 0
  speechiness = 0.035
  tempo = 84.011
  time_signature = 4
  track_href = 'https://api.spotify.com/v1/tracks/3m86qx7aIeqFcYcQ1UXUvP'
  type = 'audio_features'
  uri = 'spotify:track:3m86qx7aIeqFcYcQ1UXUvP'
  valence = 0.109

In [28]:
spotify.track_audio_features('3m86qx7aIeqFcYcQ1UXUvP')

AudioFeatures with fields:
  acousticness = 0.526
  analysis_url = 'https://api.spotify.com/v1/audio-analysis/3m86qx7aIeq...'
  danceability = 0.421
  duration_ms = 272107
  energy = 0.517
  id = '3m86qx7aIeqFcYcQ1UXUvP'
  instrumentalness = 1.82e-06
  key = 11
  liveness = 0.136
  loudness = -11.49
  mode = 0
  speechiness = 0.035
  tempo = 84.011
  time_signature = 4
  track_href = 'https://api.spotify.com/v1/tracks/3m86qx7aIeqFcYcQ1UXUvP'
  type = 'audio_features'
  uri = 'spotify:track:3m86qx7aIeqFcYcQ1UXUvP'
  valence = 0.109

### Dummy dataset

In [24]:
import sys
sys.path.append("app")
import pandas as pd
%load_ext autoreload

%autoreload 2

from spotifyData import getdata_faster
from utils import request_manager
creator = getdata_faster.DatasetCreator(spotify_api=spotify)

request_manager = request_manager.RequestManager()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [25]:
dfs = []
#phrases = ['Top 50 Australia', 'Top 50 Poland', 'Top 50 USA']
phrases = ['Funky 80\'s', 'best metal songs', 'best rock']
phrases = ["Happy mix", "best sad songs"]
for phrase in phrases:
    outcome = creator.search(phrase, type="playlist")
    rows = request_manager.add_data([], outcome, query_name=phrase)
    request_manager.add_request(phrase, "playlist")
    dfs.append(pd.DataFrame(rows))


<class 'tekore.model.ModelList'>
<class 'tekore.model.ModelList'>


In [26]:
df = pd.concat(dfs, axis=0)

In [27]:
df.to_csv("Different_modes.csv")

In [23]:
df

,artist,explicit,name,album,popularity,acousticness,danceability,duration,energy,instrumentalness,...,liveness,loudness,mode,speechiness,time_signature,valence,tempo,id,query,count
0,Mary Jane Girls,False,In My House - Single Version,80s Soul (International Version),24,0.013700,0.795,3,0.664,0.079600,...,0.0726,-9.171,minor,0.0316,4,0.936,116.808,1,Funky 80's,1
1,Michael Jackson,False,Thriller - Single Version,The Essential Michael Jackson,0,0.093400,0.775,5,0.815,0.000034,...,0.2560,-5.639,major,0.0477,4,0.835,118.402,1,Funky 80's,1
2,Change,False,A Lovers'S Holiday - Single Edit,Greatest Hits & Rare Tracks,0,0.027700,0.932,4,0.651,0.009590,...,0.0714,-5.747,minor,0.2240,4,0.821,116.357,1,Funky 80's,1
3,George Duke,False,Shine On,DREAM ON,40,0.017600,0.800,5,0.770,0.011500,...,0.0285,-11.271,minor,0.0415,4,0.960,118.832,1,Funky 80's,1
4,CHIC,False,Believer,The Best of Chic Vol. 2,21,0.360000,0.855,5,0.506,0.006870,...,0.0391,-14.156,major,0.0944,4,0.882,115.708,1,Funky 80's,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Tenacious D,True,Tribute,Tenacious D,65,0.181000,0.365,4,0.767,0.000856,...,0.0942,-6.185,minor,0.0681,4,0.422,94.174,3,best rock,1
96,Muse,False,Knights of Cydonia,Black Holes and Revelations,67,0.000273,0.366,6,0.963,0.012200,...,0.1150,-5.301,minor,0.1420,4,0.211,137.114,3,best rock,1
97,Puddle Of Mudd,False,Gimme Shelter,Re (Disc) Overed,45,0.004820,0.502,5,0.898,0.021900,...,0.1460,-4.086,minor,0.0431,4,0.456,120.015,3,best rock,1
98,Green Day,True,American Idiot,American Idiot,77,0.000026,0.380,2,0.988,0.000079,...,0.3680,-2.042,major,0.0639,4,0.769,186.113,3,best rock,1
